In [ ]:
!pip install basemap

     |████████████████████████████████| 862 kB 4.1 MB/s 
     |████████████████████████████████| 6.3 MB 41.2 MB/s 
     |████████████████████████████████| 219 kB 45.8 MB/s 
     |████████████████████████████████| 30.5 MB 1.4 MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37324 sha256=a8b2e3c86984e6b40facdfb58d6a1c0267fc366f510ea80f4d33ea8c2dd61d3a
  Stored in directory: /root/.cache/pip/wheels/43/f8/87/53c8cd41545ba20e536ea29a8fcb5431b5f477ca50d5dffbbe
Successfully built pyshp


In [ ]:
#   This is a code to optimize the weights for the radii of gyration
#   It calls the method weight_optimizer.py

import sys
import os
#import numpy as np
#from array import array
sys.path.insert(0, '/content/drive/My Drive/dop_notebooks')

import matplotlib
import numpy as np
from numpy import *
from array import array

import math
from scipy import special
#import scipy.special

import random

import datetime
from osgeo import ogr, osr

import BSTUtilities
import BSTCalcMethods
import BSTFileMethods
import BSTPlotMethods
import BSTUtilities

In [ ]:
def time_series_mean_std(values_list, weight_list):

#   This method computes the mean and standard deviation of the time series
#       at every time step using the adjusted time series weights

    mean_ts = 0.
    std_ts = 0.
    
    mean_ts_list = []
    
    for i in range(len(values_list)):
    
        mean_term = float(values_list[i])*float(weight_list[i])
        mean_ts_list.append( mean_term )
        
    mean_ts = sum(mean_ts_list)
    
    variance_ts_list = []
    
    for i in range(len(values_list)):
    
        variance_term = float(weight_list[i]) * math.pow(float(values_list[i])-float(mean_ts),2)
        variance_ts_list.append( variance_term )
            
    std_ts = math.pow(sum(variance_ts_list),0.5)

    return mean_ts, std_ts

In [ ]:
def weight_optimizer():

#   This code optimizes the combination of the radii of gyration curves using 
#       a brute force optimization

    interpolation_file_name = '/content/drive/MyDrive/dop_notebooks/interpolation_file.txt'
    interpolation_file = open(interpolation_file_name, 'r')
    
    count = len(open(interpolation_file_name).readlines(  ))
    print('')
    print('length of interpolation file: ',count)
    print('')
    
    ENF_list    =   []
    CLF_list    =   []

    radii_list  =   [[] for i in range(count)]  #   A list of lists
    
    i=0
    for line in interpolation_file:

        items = line.strip().split()
#         print(items)
#         print(items[6],items[7],items[8],items[9])
        
        ENF_list.append(items[0])
        CLF_list.append(items[1])
        
        radii_list[i].extend((items[6],items[7],items[8],items[9]))
        
        i += 1
        
#     for i in range(len(radii_list)):
#         print('radii_list: ', radii_list[i])
        
    #   print initial non-normalized weights

    initial_weights =   [1.0/float(count) for i in range(count)]     #   We start with equal weighting
    
    adjusted_weights=   [1.0/float(count) for i in range(count)]
#     
# #     print('')
# #     print('Initial Non-Normalized Weights: ', initial_weights)
# #     print('')
#     
# #     l = [random.randint(0,1) for i in range(100)]
# #     print('l: ',l)
# #     
#     
#     for kk in range(len(ENF_list)):
#     
#         random_weights = random.sample(range(0, len(initial_weights)), len(initial_weights))
#         sum_of_weights = sum(random_weights)
#         random_weights = [float(random_weights[i])/sum_of_weights for i in range(len(random_weights))]
    
    mean_radius_4 = [0. for i in range(4)]
    
    for i in range(4):
        for j in range(count):
#             mean_radius[i] += random_weights[j]*float(radii_list[j][i])
            mean_radius_4[i] += initial_weights[j]*float(radii_list[j][i])
    
    mean_radius_init = np.mean(mean_radius_4)
    mean_radius_init = round(mean_radius_init,2)
    
    std_radius_init  = np.std(mean_radius_4)
    std_radius_init = round(std_radius_init,2)
    
    mean_radius_4_init = [round(mean_radius_4[i],2) for i in range(4)]
    
    trials = 20000
    
    min_mean_radius = mean_radius_init
    min_std_radius  = std_radius_init
    
    for k in range(trials):       
    
        random_weights = random.sample(range(0, count), count)
        sum_of_weights = sum(random_weights)
        random_weights = [float(random_weights[i])/sum_of_weights for i in range(len(random_weights))]
        
        mean_radius_4 = [0. for i in range(4)]
        
        for i in range(4):

            for j in range(count):
                mean_radius_4[i] += random_weights[j]*float(radii_list[j][i])
    
        mean_radius = np.mean(mean_radius_4)
        mean_radius = round(mean_radius,2)
    
        std_radius  = np.std(mean_radius_4)
        std_radius = round(std_radius,2)
        
        mean_radius_4 = [round(mean_radius_4[i],2) for i in range(4)]
#         print('Adjusted Mean_Radii (Km): ', mean_radius_4)
#         print('Mean Adjusted Radius Value: ', mean_radius, '+/-', std_radius, 'Km)')
#         print('')
        
        if (std_radius < min_std_radius):
            min_std_radius = std_radius
            min_mean_radius = mean_radius
            min_random_weights = random_weights
            
#     print(min_random_weights)


        
    print('')
    print('------------------------------------------------')
    print('')
    print('Non-adjusted Mean_Radii (Km): ', mean_radius_4_init)
    print('')
    print('Mean Non-Adjusted Radius Value: ', mean_radius_init, '+/-', std_radius_init, 'Km')
    print('')
    print('Minimum Mean Adjusted Radius Value: ', min_mean_radius, '+/-', min_std_radius, 'Km')
    print('')
    
    for i in range(4):
        values_list = []
        
        for j in range(count):
            values_list.append(float(radii_list[j][i]))
        
        mean_ts, std_ts = time_series_mean_std(values_list, min_random_weights)
        
        print('EQ: ',i+1,' Minimum Mean Adjusted Radius Value: ', round(mean_ts,2), '+/-', round(std_ts,2), 'Km')
        print('')
    
    print('------------------------------------------------')
    print('')

    
    interpolation_file.close()

#     return initial_weights and adjusted weights

    return initial_weights, adjusted_weights
    
    #.................................................................

In [ ]:


#   -----------------------------------------------------------------

initial_weights, adjusted_weights = weight_optimizer()

# print('')
print('Initial Weights: ',initial_weights)
# print('')


length of interpolation file:  65


------------------------------------------------

Non-adjusted Mean_Radii (Km):  [1.64, 1.84, 1.79, 0.57]

Mean Non-Adjusted Radius Value:  1.46 +/- 0.52 Km

Minimum Mean Adjusted Radius Value:  1.37 +/- 0.47 Km

EQ:  1  Minimum Mean Adjusted Radius Value:  1.61 +/- 0.11 Km

EQ:  2  Minimum Mean Adjusted Radius Value:  1.72 +/- 0.47 Km

EQ:  3  Minimum Mean Adjusted Radius Value:  1.59 +/- 0.61 Km

EQ:  4  Minimum Mean Adjusted Radius Value:  0.56 +/- 0.05 Km

------------------------------------------------

Initial Weights:  [0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615384615385, 0.015384615